In [1]:
#Enter Twitter API credentials
consumer_key = "KQ6yHFrCxvqWtueT2ns0Yss5u"
consumer_secret = "9zxXNLKy1cJ66RhULnoXAx0O0RaFcAYBJQcIpP5YEU7SaRpQZs"
access_key = "404816384-QYG3ORHKly19S1rVxnQjl36pbJP1OLrD7vrtTkEk"
access_secret = "5oxR9HEFR3SfF21AMC7t1wNAdK5mUB4owZDFhnUJMUcvC"

In [2]:
import tweepy #https://github.com/tweepy/tweepy
import csv

def get_all_tweets(screen_name):
    #Twitter only allows access to a users most recent 3240 tweets with this method

    #authorize twitter, initialize tweepy
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth)
    

    alltweets = []

    #make initial request for most recent tweets (200 is the maximum allowed count)
    new_tweets = api.user_timeline(screen_name = screen_name,count=200)
    
    #save most recent tweets
    alltweets.extend(new_tweets)
    
    #save the id of the oldest tweet less one
    oldest = alltweets[-1].id - 1
    
    #keep grabbing tweets until there are no tweets left to grab
    while len(new_tweets) > 0:
        print "getting tweets before %s" % (oldest)

        #all subsiquent requests use the max_id param to prevent duplicates
        new_tweets = api.user_timeline(screen_name = screen_name,count=200,max_id=oldest)

        #save most recent tweets
        alltweets.extend(new_tweets)

        #update the id of the oldest tweet less one
        oldest = alltweets[-1].id - 1

        print "...%s tweets downloaded so far" % (len(alltweets))

    #transform the tweepy tweets into a 2D array that will populate the csv	
    outtweets = [[tweet.id_str, tweet.created_at, tweet.text.encode("utf-8")] for tweet in alltweets]
    
    #write the csv
    with open('%s_tweets.csv' % screen_name, 'wb') as f:
        writer = csv.writer(f)
        writer.writerow(["id","created_at","text"])
        writer.writerows(outtweets)
    
    pass


if __name__ == '__main__':
    get_all_tweets("houseofcoates")

getting tweets before 775332124329115647
...400 tweets downloaded so far
getting tweets before 763183849761304580
...600 tweets downloaded so far
getting tweets before 752530214886989823
...800 tweets downloaded so far
getting tweets before 716452399217512448
...999 tweets downloaded so far
getting tweets before 692820247041175551
...1199 tweets downloaded so far
getting tweets before 648212614904971264
...1399 tweets downloaded so far
getting tweets before 614153026669051903
...1599 tweets downloaded so far
getting tweets before 591092016660017151
...1799 tweets downloaded so far
getting tweets before 561710638280749056
...1999 tweets downloaded so far
getting tweets before 540154417425940479
...2199 tweets downloaded so far
getting tweets before 523965275377782783
...2399 tweets downloaded so far
getting tweets before 504721527787749375
...2599 tweets downloaded so far
getting tweets before 478283092592189440
...2799 tweets downloaded so far
getting tweets before 447221528850358271
.

After obtaining the CSV file, we proceed to extract the data

![Image](Images/fig1.jpg)

![Image](Images/fig2.jpg)

In [15]:
import pandas as pd
import numpy as np
import datetime

In [16]:
test = pd.read_csv('houseofcoates_tweets.csv')

In [52]:
date_time_list = []
for i,ele in enumerate(test.iloc[:,2]):
    if 'Someone' in ele:
        if 'off' not in ele: 
            date_time_list.append(test.iloc[i]['created_at'])
    elif 'someone' in ele:
        if 'off' not in ele: 
            date_time_list.append(test.iloc[i]['created_at'])
    elif 'welcome' in ele:
        date_time_list.append(test.iloc[i]['created_at'])
    elif 'now on' in ele:
        date_time_list.append(test.iloc[i]['created_at'])
time = pd.to_datetime(date_time_list)

dates = []
hours = []
for ele in time:
    dates.append(ele.date())
    hours.append(ele.time())
print dates[0],hours[0]

2016-09-28 16:43:16


In [9]:
for ele in test.iloc[:,2]:
    dt = test
    print dt
time = pd.to_datetime(test.iloc[:,1])

ValueError: need more than 2 values to unpack

In [14]:
for z in test.iloc[:,0]:
    print z

781172436209377280
781133059143917568
781016692868653056
781008713356304385
780978641190682624
780970529805234176
780942746202804224
780919345656520704
780916664460251136
780914636417892353
780914081163276288
780889574755237888
780796228888096768
780796214187098112
780770649367347200
780652716687233024
780580347775942656
780545301165793280
780545001130524672
780544999142334464
780544620090560512
780543653970374657
780445335579754496
780408034728275968
780323290145492992
780307967912607744
780257720045797376
780225999527174145
780215143779143680
780187027643985920
780186904499261440
780186181288034304
780151035688054784
780139962276704256
780126741092245504
780069234906988545
780068275413712896
780044763277959172
779863615545479169
779812454935175168
779801424792260608
779795087039553536
779792325912526850
779711475598364672
779567207382736896
779553645981499392
779552764535926784
779536716579348480
779523822429016064
779523810596970496
779523754670039040
779523697036103680
779508695403